In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets -q

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer

from datasets import load_dataset
from sentence_transformers import SentenceTransformer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cud

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
MODEL_NAME = "freelawproject/modernbert-embed-base_finetune_8192"
MAX_TOKENS = 8192
MAX_SENTENCE = int(0.002 * MAX_TOKENS)

MODEL = SentenceTransformer(MODEL_NAME)
TOKENIZER = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/82.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/596M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [4]:
MAX_SENTENCE

16

In [5]:
def split_text_into_chunks(text, tokenizer=TOKENIZER, max_tokens=MAX_TOKENS, num_overlap_sentences=MAX_SENTENCE) -> list[str]:
    """Split text into chunks based on sentences, not exceeding max_tokens, with sentence overlap"""

    # Split the text to sentences & encode sentences with tokenizer
    sentences = sent_tokenize(text)
    encoded_sentences = [
        tokenizer.encode(sentence, add_special_tokens=False)
        for sentence in sentences
    ]
    lead_text = "search_document: "
    lead_tokens = tokenizer.encode(lead_text)
    lead_len = len(lead_tokens)
    chunks = []
    current_chunks: list[str] = []
    current_token_counts = len(lead_tokens)

    for sentence_tokens in encoded_sentences:
        sentence_len = len(sentence_tokens)
        # if the current sentence itself is above max_tokens
        if lead_len + sentence_len > max_tokens:
            # store the previous chunk
            if current_chunks:
                chunks.append(lead_text + " ".join(current_chunks))
            # truncate the sentence and store the truncated sentence as its own chunk
            truncated_sentence = tokenizer.decode(
                sentence_tokens[: (max_tokens - len(lead_tokens))]
            )
            chunks.append(lead_text + truncated_sentence)

            # start a new chunk with no overlap (because adding the current sentence will exceed the max_tokens)
            current_chunks = []
            current_token_counts = lead_len
            continue

        # if adding the new sentence will cause the chunk to exceed max_tokens
        if current_token_counts + sentence_len > max_tokens:
            overlap_sentences = current_chunks[
                -max(0, num_overlap_sentences) :
            ]
            # store the previous chunk
            if current_chunks:
                chunks.append(lead_text + " ".join(current_chunks))

            overlap_token_counts = tokenizer.encode(
                " ".join(overlap_sentences), add_special_tokens=False
            )
            # If the sentence with the overlap exceeds the limit, start a new chunk without overlap.
            if (
                lead_len + len(overlap_token_counts) + sentence_len
                > max_tokens
            ):
                current_chunks = [tokenizer.decode(sentence_tokens)]
                current_token_counts = lead_len + sentence_len
            else:
                current_chunks = overlap_sentences + [
                    tokenizer.decode(sentence_tokens)
                ]
                current_token_counts = (
                    lead_len + len(overlap_token_counts) + sentence_len
                )
            continue

        # if within max_tokens, continue to add the new sentence to the current chunk
        current_chunks.append(tokenizer.decode(sentence_tokens))
        current_token_counts += len(sentence_tokens)

    # store the last chunk if it has any content
    if current_chunks:
        chunks.append(lead_text + " ".join(current_chunks))
    return chunks

In [6]:
file_path = '/content/drive/MyDrive/similar_cases/docket_progression.csv'

df = pd.read_csv(file_path)
df.head()

,docket_id,summary
0,119293,For docket number 12-3705 of case United State...
1,1088363,For docket number 17-60417 of case Cerrell Dar...
2,3050501,"For docket number 2015-1767, 2015-1768 of case..."
3,4239842,For docket number 16-10572 of case Connie Lee ...
4,4246017,For docket number 1:04-mj-00312 of case United...


In [7]:
for index, row in df.iterrows():
  chunks = split_text_into_chunks(row["summary"])
  embeddings = MODEL.encode(chunks)
  mean_embedding = np.mean(embeddings, axis=0)
  max_embedding = np.max(embeddings, axis=0)
  df.at[index, "mean_embedding"] = mean_embedding.astype(object)
  df.at[index, "max_embedding"] = max_embedding.astype(object)

df.head()

W0404 00:27:32.142000 1945 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode


,docket_id,summary,mean_embedding,max_embedding
0,119293,For docket number 12-3705 of case United State...,"[0.015109852887690067, -0.05136959254741669, 0...","[0.0297186691313982, -0.0062833307310938835, 0..."
1,1088363,For docket number 17-60417 of case Cerrell Dar...,"[-0.015262693166732788, 0.014277754351496696, ...","[-0.015262693166732788, 0.014277754351496696, ..."
2,3050501,"For docket number 2015-1767, 2015-1768 of case...","[0.011641735211014748, 0.016404390335083008, 0...","[0.011641735211014748, 0.016404390335083008, 0..."
3,4239842,For docket number 16-10572 of case Connie Lee ...,"[-0.002721260767430067, 0.07355376332998276, 0...","[-0.002721260767430067, 0.07355376332998276, 0..."
4,4246017,For docket number 1:04-mj-00312 of case United...,"[-0.011532357893884182, -0.020075136795639992,...","[-0.011532357893884182, -0.020075136795639992,..."


In [8]:
df["max_embedding"] = df["max_embedding"].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)
df["mean_embedding"] = df["mean_embedding"].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

file_path = '/content/drive/MyDrive/similar_cases/8192_embeddings.json'
df.to_json(file_path, orient="records", indent=2)